# Introduction

Perform a Binary Classification over the Rocket League score chance of Team A and Team B.

In [1]:
# Import Standard Libraries
import pandas as pd

In [ ]:
# Notebook's variables


# Read Data

Since the data are to big for being all read, a 20% of sample data from each file would be taken into account and concatenated into a single DataFrame.

In [13]:
# Read train dtypes
train_dtypes = pd.read_csv('../input/tabular-playground-series-oct-2022/train_dtypes.csv')

train_data = pd.read_csv('../input/tabular-playground-series-oct-2022/train_0.csv', 
                         dtype=dict(zip(train_dtypes.column, train_dtypes.dtype)), 
                         nrows=10000)

In [14]:
train_data.head()

,game_num,event_id,event_time,ball_pos_x,ball_pos_y,ball_pos_z,ball_vel_x,ball_vel_y,ball_vel_z,p0_pos_x,...,boost0_timer,boost1_timer,boost2_timer,boost3_timer,boost4_timer,boost5_timer,player_scoring_next,team_scoring_next,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,1,1002,-33.313030,-0.0,0.0,1.8548,-0.0,0.0,0.0,41.804798,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
1,1,1002,-33.206146,-0.0,0.0,1.8548,-0.0,0.0,0.0,42.261600,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
2,1,1002,-33.098114,-0.0,0.0,1.8548,-0.0,0.0,0.0,43.227001,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
3,1,1002,-32.993191,-0.0,0.0,1.8548,-0.0,0.0,0.0,43.898399,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
4,1,1002,-32.887756,-0.0,0.0,1.8548,-0.0,0.0,0.0,44.960602,...,0.0,0.0,0.0,0.0,0.0,0.0,3,B,0,0
